In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

We will code **Capsule Networks** in PyTorch. We will mainly be working with MNIST dataset only in this 

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torchvision import transforms, datasets

In [0]:
import numpy as np

def one_hot_transform(labels):
    one_hot = torch.zeros(labels.size()+tuple([10])).scatter_(-1, labels, 1)
    return one_hot 
  
trainset = datasets.MNIST ( root='../data', train = True, download = True, transform = transforms.Compose([ transforms.Pad(2), 
                                                                                                            transforms.RandomCrop(28),
                                                                                                            transforms.ToTensor() ]),
                            target_transform = one_hot_transform)

testset = datasets.MNIST ( root='../data', train = False, download = True, transform = transforms.Compose([ transforms.ToTensor() ]),
                            target_transform = one_hot_transform)


train_loader = torch.utils.data.DataLoader( trainset, batch_size = 32, shuffle = True, num_workers = 4)

test_loader = torch.utils.data.DataLoader( testset, batch_size = 32, shuffle = False, num_workers = 4)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
class CapsLayer(nn.Module):
    def __init__(self, in_channels, out_channels, num_iterations=3, num_capsules=1,  num_route_nodes=-1, primary=False, 
                  caps_dim=-1, kernel_size=None, stride=None):
        
        super(CapsLayer, self).__init__()
        
        self.primary = primary
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        if primary:
            self.caps_dim = caps_dim
            self.kernel_size = kernel_size
            self.stride = stride
            self.caps = nn.Conv2d(in_channels, out_channels, kernel_size = self.kernel_size, stride = self.stride, padding=0)
        else:
            self.num_capsules = num_capsules
            self.num_iterations = num_iterations
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        
    def squash(self, x, dim):
        squared_norm = (x**2).sum(dim = dim, keepdim = True)
        return (squared_norm / (1+squared_norm)) * x / (squared_norm**0.5)
        
    def forward(self, x):
        if not self.primary:
            U = x[:, None, :, None, :] @ self.route_weights[None, :, :, :, :]
            B = torch.zeros(U.size()).to(device) #the first device one
            for i in range(self.num_iterations):
                C = F.softmax(B, dim=2)
                V = (C*U).sum(dim=2, keepdim=True)
                V = self.squash(V, dim=-1)
                B = B+(U*V).sum(dim=-1, keepdim=True).sum(dim=-2, keepdim=True)
            V = V.view(-1, V.size(1), V.size(4))
        else:
            V = [self.caps(x).view(x.size(0), -1, 1) for _ in range(self.caps_dim)]
            V = torch.cat(V, dim=-1)
            V = self.squash(V, dim=-1)
        return V

In [0]:

class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 256, kernel_size = 9, stride = 1, padding = 0)
        
        self.primary = CapsLayer(in_channels = 256, out_channels = 32, primary = True, kernel_size = 9, stride = 2, caps_dim = 8)
        
        self.digital = CapsLayer(in_channels = 8, out_channels = 16, num_capsules = 10, num_route_nodes = 32*6*6)
        
        self.reconst = nn.Sequential( nn.Linear(16*10 ,512),
                                      nn.ReLU(inplace = True),
                                      nn.Linear(512, 1024),
                                      nn.ReLU(inplace = True),
                                      nn.Linear(1024, 784),
                                      nn.Sigmoid()
                                    )
    
    def reconstruction(self, x, y):
        recon_images = self.reconst((x*y[:,:,None]).view(-1, 160)).view(-1, 28, 28)
        return recon_images

    def forward(self, x, y):
        l1=F.relu(self.conv1(x))
        l2=self.primary(l1)
        l3=self.digital(l2)
        
        preds=F.softmax((((l3**2).sum(dim=-1))**(0.5)), dim=-1)
        
        return preds, self.reconstruction(l3, y)

In [0]:
class CapsLoss(nn.Module):
    def __init__(self, reconstrn_weight = 0.000):
        super(CapsLoss, self).__init__()
        self.reconstrn_weight = reconstrn_weight
    def forward(self, images, labels, preds, reconst_images):
        preds = torch.clamp(preds, min=0.1, max=0.9)
        margin_loss = (labels*((0.9-preds)**2) + 0.5*(1-labels)*((preds-0.1)**2)).sum()
        reconst_loss = ((images-reconst_images)**2).sum()
        return (margin_loss + self.reconstrn_weight * reconst_loss) / images.size(0)

In [0]:
import torch.optim as optim

criterion = CapsLoss()
net = CapsNet().to(device) #for GPU
optimizer = optim.Adam(net.parameters(), lr = 0.0001)

for epoch in range(30):  # fix num-epochs as 30.

    # training
    net.train()
    running_loss = 0.0
    count = 0.0
    for data in train_loader:
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        inputs = (inputs-127.5)/128
        outputs, reconst = net(inputs, labels)
        loss = criterion(inputs, labels, outputs, reconst)
        
        loss.backward()
        optimizer.step()

        # statistics
        running_loss +=  ( loss.item() * inputs.size(0)) #multiply by the batch size
        count += sum(np.argmax(labels.data.cpu().numpy(), 1) == np.argmax(outputs.data.cpu().numpy(), 1)) #the accuracy
        
    print ( "epoch num: %d, Train loss: %.5f, Training Accuracy is: %.5f " % (epoch + 1, running_loss/len(train_loader), count/len(train_loader)) )
    
    # testing
    net.eval()
    running_loss = 0.0
    count = 0.0
    for data in test_loader:
        # get the inputs
        inputs, labels = data
        
        inputs = (inputs-127.5)/128
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward 
        with torch.no_grad():
            outputs, reconst = net(inputs, labels)
        loss = criterion(inputs, labels, outputs, reconst)

        # statistics
        running_loss +=  ( loss.item() * inputs.size(0)) #multiply by the batch size
        count += sum(np.argmax(labels.data.cpu().numpy(), 1) == np.argmax(outputs.data.cpu().numpy(), 1)) #the accuracy
        
    print ( "epoch num: %d, Test loss: %.5f, Testing Accuracy is: %.5f " % (epoch + 1, running_loss/len(test_loader), count/len(test_loader)) )